In [1]:
import tensorflow_hub as hub

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


2023-06-13 18:13:32.672155: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-13 18:13:32.674722: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-13 18:13:32.736830: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-13 18:13:32.737907: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 18:13:33.818133: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
df = pd.read_csv('spam.csv', encoding='latin-1')


In [3]:
df.describe()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""","MK17 92H. 450Ppw 16""","GNT:-)"""
freq,4825,30,3,2,2


In [4]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)

In [6]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
df.groupby('v1').describe()

v2                                                               
     count unique                                                top freq
v1                                                                       
ham   4825   4516                             Sorry, I'll call later   30
spam   747    653  Please call our customer service representativ...    4

In [8]:
# data imbalance

In [9]:
df1 = df[df.v1 == 'ham']
df2 = df[df.v1 =="spam"]
df1.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
6,ham,Even my brother is not like to speak with me. ...


In [10]:
df1.shape 

(4825, 2)

In [11]:
df3 = df1.sample(df2.shape[0])
df3.shape

(747, 2)

In [12]:
df3.head()

,v1,v2
4213,ham,Ard 530 like dat lor. We juz meet in mrt stati...
110,ham,What is the plural of the noun research?
227,ham,Hey company elama po mudyadhu.
185,ham,Hello handsome ! Are you finding that job ? No...
1182,ham,"K give me a sec, breaking a &lt;#&gt; at cstore"


In [13]:
df4 = pd.concat([df3, df2])
df4.head()

,v1,v2
4213,ham,Ard 530 like dat lor. We juz meet in mrt stati...
110,ham,What is the plural of the noun research?
227,ham,Hey company elama po mudyadhu.
185,ham,Hello handsome ! Are you finding that job ? No...
1182,ham,"K give me a sec, breaking a &lt;#&gt; at cstore"


In [14]:
df4.shape

(1494, 2)

In [15]:
df4 = pd.DataFrame.sample(df4,frac=1)

In [16]:
y = df4["v1"].apply(lambda x : 1 if x == "spam" else 0)
x = df4["v2"]

df4.head()



,v1,v2
1364,ham,Hey i will be really pretty late... You want t...
249,spam,Congratulations ur awarded 500 of CD vouchers ...
524,spam,"URGENT!: Your Mobile No. was awarded a å£2,000..."
3414,ham,He remains a bro amongst bros
3562,spam,Auction round 4. The highest bid is now å£54. ...


In [17]:
x

1364    Hey i will be really pretty late... You want t...
249     Congratulations ur awarded 500 of CD vouchers ...
524     URGENT!: Your Mobile No. was awarded a å£2,000...
3414                        He remains a bro amongst bros
3562    Auction round 4. The highest bid is now å£54. ...
                              ...                        
4680                            Are you staying in town ?
796     Orange customer, you may now claim your FREE C...
3690    Sir, i am waiting for your call, once free ple...
5457                          Arun can u transfr me d amt
540     from www.Applausestore.com MonthlySubscription...
Name: v2, Length: 1494, dtype: object

In [18]:
y

1364    0
249     1
524     1
3414    0
3562    1
       ..
4680    0
796     1
3690    0
5457    0
540     1
Name: v1, Length: 1494, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [20]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=0)

# try stratify

In [44]:
import tensorflow_hub as tfhub
import tensorflow as tf

preprocess_url1 = "bert_en_uncased_preprocess_3/saved_model.pb"
encoder_url1 = "bert_en_uncased_L-24_H-1024_A-16_4/saved_model.pb"



bert_preprocess_layer = tf.saved_model.load(preprocess_url1)
bert_encoder_layer = tf.saved_model.load(encoder_url1)





OSError: SavedModel file does not exist at: bert_en_uncased_preprocess_3/saved_model.pb/{saved_model.pbtxt|saved_model.pb}

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
testt = ["banana","apple","grapes","elon musk","google"]

result = bert_preprocess(bert_preprocess_layer(testt))["pooled_output"]
cosine_similarity([result[0],result[1]])

NameError: name 'bert_preprocess' is not defined

In [27]:
import tensorflow as tf
from tensorflow.keras.layers import Input,Dense,Dropout
from tensorflow.keras.models import Model


In [28]:
input = Input(shape=(),dtype=tf.string,name="text")
preprocessed_text = bert_preprocess_layer(input)
result_layer = bert_encoder_layer(preprocessed_text)

la = Dropout(0.3)(result_layer["pooled_output"])

la = Dense(1,activation="sigmoid",name = "output")(la)
model_func = Model(input , la)



NameError: name 'bert_preprocess_layer' is not defined

In [29]:
model_func.summary()

NameError: name 'model_func' is not defined

In [30]:
model_func.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model_func.fit(x_train,y_train,epochs=10,validation_split = 0.1)

NameError: name 'model_func' is not defined

In [35]:
model_func.evaluate(x_test,y_test)

NameError: name 'model_func' is not defined

In [36]:
y_predicted = model_func.predict(x_test)
y_predicted = y_predicted.flatten()

NameError: name 'model_func' is not defined

In [37]:
y_predicted  = np.where(y_predicted>0.5,1,0)

NameError: name 'y_predicted' is not defined

In [34]:
from sklearn.metrics import confusion_matrix,classification_report

cm = confusion_matrix(y_test,y_predicted)

NameError: name 'y_predicted' is not defined

In [38]:
import seaborn as sns
sns.heatmap(cm,anoot = True,fmt = "d")
plt.xlabel("predicted")
plt.ylabel("truth")

NameError: name 'cm' is not defined

In [39]:
print(classification_report(y_test,y_predicted))

SyntaxError: unexpected EOF while parsing (1228285558.py, line 1)